# Tracking

Kevin J. Walchko

created 28 Oct 2017

---

## References

- [Object tracking with kalman filter](https://www.codeproject.com/Articles/865935/Object-Tracking-Kalman-Filter-with-Ease)
- [Kalman filter for dummies](http://bilgin.esme.org/BitsAndBytes/KalmanFilterforDummies)

## Setup

In [1]:
%matplotlib inline 

import cv2         # opencv itself
import numpy as np # matrix manipulations

from matplotlib import pyplot as plt

A common use for computer vision is detection and tracking of various objects through a scene. Typically there is a process flow of:

1. clean up and calibrate image
1. update
    1. if *not* already tracking an object
        1. then search the entire image and try to detect an object to track
    1. if already tracking an object
        1. based off the predicted location, detect the object within a window around the predicted location and update
        1. if object not found, search entire image and update
        1. if the object is not found, *esitmate* where the object should have been
1. Using the new location, predict where the target will be in the future

There are several issues which make tracking difficult:

- target object initialization
- occlusions which hide the target object
- tracking multiple objects and difficulties that arise if they cross paths
- scale, illumination and change of appearance
- objects that have difficult and rapid motions

Even though object tracking has been a problem for many years, it is still not solved despite the fact that there are many object trackers, even the ones built for special purposes, and those which want to solve the problem in general.

Kalman filters, although they can be used for many other purposes, can be used for object tracking. They are especially convenient for objects which motion model is known, plus they incorporate some extra information in order to estimate the next object position more robustly. Unfortunately they are only useful for tracking single targets reliably.

## Kalman Filter - Main Idea

Let's assume that we have some kind of detector and that our detector is imperfect: it is prone to false positives, does not always detect objects, detects them imperfectly (does not provide exact position and scale) and its execution is costly. Let us also assume that we are tracking a single football player. After we detect the object, we want to leverage our information about the object in order to track it as robustly as we can. Our detector will give us just the location of the object so we have to use it as best as possible. In order to predict the next position of the player, we will need an object motion model (e.g. constant velocity motion, constant acceleration motion). Our detector is imperfect so there is noise in object locations, generally called measurement noise. Also, a selected motion model will not describe our player motion perfectly, so we also have noise regarding the model, called process noise. We want to estimate the next player position incorporating only the three parameters:

- Object motion model
- Measurement noise
- Process noise

So, we could efficiently re-detect it and hopefully cope with object occlusion. These next sections will show you how to do it.

<img src="pics/cycle.png" width="500px">

## Initial State

We are going to first introduce the initial state and what we are trying to accomplish. The following figures show the initial state and in the same time introduce most relevant terms.

<img src="pics/initial-1.png" width="500px">

**Overview:** Using only estimates and the current state, we want to predict the next state. The second step (correction) includes a noisy measurement in order to apply a state update.

<img src="pics/initial-2.png" width="500px">

**Initial state type:** The green line at the top represents an object we’d like to track, with the blue X’s marking the object's true position. We want to model motion by using a constant velocity model. Therefore, the state will include the object position and velocity in both directions. The detector gives us the noisy object's position, so the position is our measurement.

<img src="pics/initial-3.png" width="500px">

**Initial state value:** In order to start the tracking process, we need to know the initial state x0|0 value. We also need to have the initial uncertainty which is expressed by Gaussian covariance matrix P0|0. The orange ellipse on the top represents the 2D Gaussian which describes the uncertainty in position. The initial matrix P0|0 is usually diagonal (assuming the components are not correlated), where the each component has its own uncertainty L - Gaussian sigma.

Reading this section bear in mind:

- x(t) - state vector
- z(t) - measurement
- P(t|t-1) - process covariance matrix

## Predict

Prediction is the first step which includes the next state (position and velocity) prediction as well as updating our uncertainty about the object state (increasing the uncertainty).

<img src="pics/predict-1.png" width="500px">

1a) State prediction: The first step is to predict the next state by using the motion model. The next state x(t|t-1) is obtained by multiplying the previous state by the state transition matrix - F. Q denotes process noise, which must follow normal distribution since we are working with Gaussians.

<img src="pics/predict-2.png" width="500px">

1b) Covariance prediction: The covariance update is done by multiplying the covariance matrix from the previous iteration by the state transition matrix F (motion model) and by adding the process noise Q which can be constant. The update covariance of our prediction is wider because we are less sure about our estimate.

<img src="pics/predict-3.png" width="500px">

* State transition matrix: The motion model must be represented by matrix F, therefore it must be linear. If the model is not linear the model must be linearized in some working point, which is used in the Extended Kalman Filter. The used model models the constant 2D velocity motion model where the position is updated as: p(t) = p(t-1) + v * p(t-1) where p denotes position and v velocity; the velocity remains constant. Velocity is marked as derivative of position in time. If we had done the prediction step multiple times, the estimated positions would follow the constant velocity model (blue dots). The covariance matrix would get wider each time (blue ellipse) as our uncertainty about the object position grows.

Reading this section, bear in mind:

- F - state transition matrix
- Q(t) - process noise covariance

## Correct

After the noisy measurement has been obtained, the correction step begins. It incorporates a Kalman filter update which includes a state update and uncertainty update (decreasing the uncertainty).

<img src="pics/correct-1.png" width="500px">

* Measure: When (if) we receive a noisy measurement (in our case position obtained form a detector), the update process begins. The noisy measurement z(t) is modeled as a single Gaussian, where the noise is modeled as covariance matrix R(t)which is usually constant. The uncertainty of the measurement appears as golden ellipse.

<img src="pics/correct-2.png" width="500px">

2a) Measurement update: In order to calculate the predicted measurement needed for correction, we must select the measurement components from the state. A measurement has the same structure as a state or it just contains state parts; in our case, just the object position. Matrix H is the model selection matrix that when multiplied with state selects only elements that belong to a measurement.

<img src="pics/correct-3.png" width="500px">

* Residual: In order to make correction, we must know the prediction error. Therefore the residual, also known as innovation, is calculated, denoted by y(t). The residual is calculated by differencing the predicted measurement and obtained measurement - see the green and golden ellipse. Residual covariance S is calculated in a similar way.

<img src="pics/correct-4.png" width="500px">

* Kalman gain: Kalman gain K specifies how much we believe the prediction vs. how much we believe in the measurement. It is a product of predicted process covariance matrix P the observation model H and inverse residual covariance S. Let us study two extreme cases:

- We are sure about measurements (we believe our detector)
If this is the case, the measurement noise matrix R is very small which results the K decreases and the measurements are weighted more heavily than prediction.
- We are sure about prediction (we believe in our motion model, no so much in the detector)
If this is the case, the measurement noise matrix R is very large, which results in the K increases and the measurements are weighted much less than the prediction.

<img src="pics/correct-5.png" width="500px">

2b) Correct: The Kalman gain is now used to update the state xand covariance matrix P as shown on the figure. The result is the updated position which is denoted by the golden ellipse.

<img src="pics/correct-6.png" width="500px">

* Final step When the correction step is finished, the next step is, again, the prediction step. Those two steps are iteratively run in order to track an object as shown on the figure.

Reading this section, you should know what is:

- R(t) - measurement noise

Now, when you know the basics, it is time for real samples, but first the brief introduction to the implementation.

# Implementation

A simple set of equations to track an object though a series of frames is:

$$
X_k = \Phi X_{k-1} + N_{process} \\
Z_k = H X_k + N_{measurement} \\
X_k = \begin{bmatrix}
  x_k, y_k, v_{xk}, v_{yk}
\end{bmatrix}^T \\
Z_k = \begin{bmatrix}
  z_x & z_y
\end{bmatrix}^T \\
\Phi = \begin{bmatrix}
  1 & 0 & \delta t & 0 \\
  0 & 1 & 0 & \delta t \\
  0 & 0 & 1 & 0 \\
  0 & 0 & 0 & 1 \\
\end{bmatrix} \\
H = \begin{bmatrix}
  1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 
\end{bmatrix}
$$

These kinematics are designed to say, an object should move smoothly from
pixel to pixel and not randomly jump around on an image plane. We are only able to directly measure an object's position (x,y). The process noise is assumed to be $N(0,Q)$ and the measure noise is assumed to be $N(0,R)$ which are both gaussian.

```python
# setup kf
kalman = cv2.KalmanFilter(4, 2, 0)  # state size, measurement size, control vector

dt = 1.
A = np.eye(4, dtype=np.float32)
A[0, 2] =  dt
A[1, 3] =  dt
kalman.transitionMatrix = A

H = np.array([[1., 0, 0, 0], [0, 1., 0, 0]], dtype=np.float32)
kalman.measurementMatrix = H

kalman.processNoiseCov = 1e-5 * np.eye(4, dtype=np.float32)
kalman.measurementNoiseCov = 1e-1 * np.eye(2, dtype=np.float32)

# initialize the kf state somehow
kalman.statePre = np.array([x, y, vx, vy], dtype=np.float32)

while(True):
    measurement = grab_sample()  # measure something
    kalman.correct(measurement)  # update kf with current measurement
    prediction = kalman.predict()  # Computes a predicted state
```

# Blob Detection

- [ref](https://www.learnopencv.com/blob-detection-using-opencv-python-c/)

OpenCV has a simple blob detector built into it. The steps are:

1. Convert the source image to binary images by applying thresholding with several thresholds from minThreshold (inclusive) to maxThreshold (exclusive) with distance thresholdStep between neighboring thresholds.
1. Extract connected components from every binary image by findContours and calculate their centers.
1. Group centers from several binary images by their coordinates. Close centers form one group that corresponds to one blob, which is controlled by the minDistBetweenBlobs parameter.
1. From the groups, estimate final centers of blobs and their radiuses and return as locations and sizes of keypoints.

The detector performs several filtrations of the discovered blobs. Available filtrations:

- By color (**This seems to be broken**). This filter compares the intensity of a binary image at the center of a blob to blobColor. If they differ, the blob is filtered out. 
    - blobColor = 0 to extract dark blobs
    - blobColor = 255 to extract light blobs.
- By area. Extracted blobs have an area between minArea (inclusive) and maxArea (exclusive).
- By circularity. Extracted blobs have circularity ($\frac{4∗\pi∗Area}{perimeter^2}$) between minCircularity (inclusive) and maxCircularity (exclusive).
    - A circle has a circularity of 1
- By ratio of the minimum inertia to maximum inertia. Extracted blobs have this ratio between minInertiaRatio (inclusive) and maxInertiaRatio (exclusive).
- By convexity. Extracted blobs have convexity ($\frac{area}{area_{blob-convex-hull}}$) between minConvexity (inclusive) and maxConvexity (exclusive).

Default values of parameters are tuned to extract dark circular blobs. This, if you calculate a mask image, usually the object of interest is white, you will need to invert the image.

```python
params = cv2.SimpleBlobDetector_Params()

# color - doesn't seem to work
# params.blobColor = 255

# Change thresholds - levels
params.minThreshold = 0  # inclusive
params.maxThreshold = 1  # exclusive
params.thresholdStep = 1

# Filter by Area.
params.filterByArea = True
params.minArea = 1500

# Filter by Circularity - doesn't seem very precise
params.filterByCircularity = True
params.maxCircularity = 1.2
params.minCircularity = .7

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.87

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(gray)
im_with_keypoints = cv2.drawKeypoints(
    frame,
    keypoints,
    None,  # output image
    (0,0,255),
    cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.imshow(im_with_keypoints)
```